In [2]:
import pandas as pd
import numpy as np
import funcs, gc, sqlite3

In [3]:
# bos ou rat
tipo = 'bos'

In [4]:
unclassif = funcs.get_unclassified_registers(tipo=tipo)
unclassif

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR,CIA
0,2020-025629373-001,DIVINOPOLIS,PERNAMBUCO,PERNAMBUCO,,,,,other,other
1,2020-021323080-001,DIVINOPOLIS,A,A,,,,,other,other


In [5]:
classificadores = funcs.get_df_classif()
unclassif.loc[:,'SETOR'] = unclassif.apply(lambda row: funcs.classifica_setor(row, classificadores), axis=1)
funcs.classifica_cia(unclassif)

unclassif

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR,CIA
0,2020-025629373-001,DIVINOPOLIS,PERNAMBUCO,PERNAMBUCO,,,,,PLANALTO,139 CIA
1,2020-021323080-001,DIVINOPOLIS,A,A,,,,,NITEROI,142 CIA


In [6]:
new_classif_dict = [
    {
        'nrat': unclass_list[0],
        'setor': unclass_list[-2],
        'cia': unclass_list[-1]
    }
    for unclass_list in unclassif.values
]

new_classif_dict

[{'nrat': '2020-025629373-001', 'setor': 'PLANALTO', 'cia': '139 CIA'},
 {'nrat': '2020-021323080-001', 'setor': 'NITEROI', 'cia': '142 CIA'}]

In [7]:
query_update = '''
UPDATE tbl_{}
    SET "SETOR" = ?,
    "CIA" == ?
WHERE
    "RAT.NUM_ATIVIDADE" = ?
'''.format(tipo)

with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    for dict_class in new_classif_dict:
        cursor.execute(query_update, (dict_class['setor'], dict_class['cia'], dict_class['nrat']))
        conn.commit()
conn.close()